<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Audio_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q transformers==4.24.0

In [2]:
! pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
! pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3_wc89l9
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-3_wc89l9
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175324 sha256=b3acfd520e0c346fb64bb26bcf104f24740142401f60636a29d2d92cafd733b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-zzw_77id/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper


In [3]:
import transformers
transformers.__version__

'4.24.0'

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download youtube video

In [3]:
! pip install youtube-dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.3 MB/s 


In [1]:
from __future__ import unicode_literals
import youtube_dl
#https://www.youtube.com/watch?v=srSNZlbhCwk&t=3s

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'outtmpl':"." + '/video.%(ext)s',
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=srSNZlbhCwk&t=3s'])
    
absolute_path = "output.wav" #file name of your downloaded audio

[youtube] srSNZlbhCwk: Downloading webpage
[download] Destination: ./video.webm
[download] 100% of 763.32KiB in 00:14
[ffmpeg] Destination: ./video.wav
Deleting original file ./video.webm (pass -k to keep)


In [27]:
from scipy.io import wavfile
import numpy as np

data = wavfile.read("/content/video.wav")
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate

In [28]:
framerate

44100

In [29]:
print(f"time {len(sounddata)/framerate} seconds")

time 256.92879818594105 seconds


In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")   

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import librosa
speech, _ = librosa.load("/content/video.wav",sr=16000)

In [ ]:
import torch
input_values = tokenizer(speech, return_tensors="pt").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

In [25]:
transcription

'YE BATAKI HEDGE A MEBARASALKATA MAPNI MOM GISAT I BANANI GI HANGIATA MARY MARMADI CA USA ME I COMPLUTETAT IS MA BOSARI GIM I TO MAMAGATI HI GIM KININ ARRAT OGE OSA BI PATI GI GA MECHADI GAN GANI ILI GAS IF I MU DI CHODI I TUGI MODI COMPUTEN MA GIM KINATAT A BE MADA MARMAGI CAMODI COLAAKUN GI CARKUPAN GERHOGE AN CAN MAYI TARAKAWAE A PASS ME HI HE AGO A LI I MAGA BONIPAS POTA TO MANA DICAKI CA GIVSITIS BETTI WHO HEV MAN OSUPUTA APKONO THOS NEPNI CHAKA DI CAI DO GI CAWI THE A MITI TA BE MANIN O DISKI GIO BA PI TA TEVA HAS SET TO NAN BATALCIN BA TE BAKTE MA BIROSO GAT JO MADDE AQULI COMANANIGI CA BETA WHO DE ASTA ME BUTTAS ALAKI VOTAKI'

In [6]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration,WhisperFeatureExtractor
import librosa

speech, _ = librosa.load("/content/video.wav",duration=56.8,sr=48000)

processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "en", task = "transcribe")
input_features = processor(speech, return_tensors="pt").input_features 
predicted_ids = model.generate(input_features,max_length=3000)
transcription = processor.batch_decode(predicted_ids)

print(processor.batch_decode(predicted_ids, skip_special_tokens = True))

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[' I']


In [12]:
speech

array([0.        , 0.        , 0.        , ..., 0.00195312, 0.00198364,
       0.0019989 ], dtype=float32)

In [10]:
input_features

tensor([[[-0.5945, -0.5945, -0.5945,  ...,  0.9885,  1.0717,  0.9540],
         [-0.5945, -0.5945, -0.5945,  ...,  1.0158,  1.1478,  0.8460],
         [-0.5945, -0.5945, -0.5945,  ...,  0.9025,  1.1163,  0.7533],
         ...,
         [-0.5945, -0.5945, -0.5945,  ..., -0.5945, -0.5945, -0.5945],
         [-0.5945, -0.5945, -0.5945,  ..., -0.5945, -0.5945, -0.5945],
         [-0.5945, -0.5945, -0.5945,  ..., -0.5945, -0.5945, -0.5945]]])

In [8]:
from transformers import WhisperFeatureExtractor
feature = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")

In [15]:
feature(speech)['input_features']

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[array([[-0.59445024, -0.59445024, -0.59445024, ...,  0.9884978 ,
          1.0717466 ,  0.9539778 ],
        [-0.59445024, -0.59445024, -0.59445024, ...,  1.0157536 ,
          1.1478286 ,  0.8460471 ],
        [-0.59445024, -0.59445024, -0.59445024, ...,  0.90251327,
          1.1162922 ,  0.75329477],
        ...,
        [-0.59445024, -0.59445024, -0.59445024, ..., -0.59445024,
         -0.59445024, -0.59445024],
        [-0.59445024, -0.59445024, -0.59445024, ..., -0.59445024,
         -0.59445024, -0.59445024],
        [-0.59445024, -0.59445024, -0.59445024, ..., -0.59445024,
         -0.59445024, -0.59445024]], dtype=float32)]

## Transcribe text from Audio

In [ ]:
import whisper
model = whisper.load_model("large")
options = dict(language='en', beam_size=5, best_of=5)
#transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)
translation = model.transcribe("/content/video.wav", **translate_options)

100%|██████████████████████████████████████| 2.87G/2.87G [00:22<00:00, 136MiB/s]
/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
translation

{'text': " This happened when I was 12 years old, I went to my grandmother's house with my mom My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer, then my uncle called me that his car has been punctured There is a tire in the house, we are close, come with the tire When I went to him, I saw that a strange thing was sitting, I asked him who are you Then he showed his face, which was very scary, then I noticed that he was just up to the stomach I ran immediately from there, but I fainted while running, when my eyes opened, I was at my grandmother's house I still don't know what it was",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 5.4,
   'text': " This happened when I was 12 years old, I went to my grandmother's house with my mom",
   'tokens': [639,
    2011,
    562,
    286

In [ ]:
translation['text']

" This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was."

## Save the Automatic Speech Recognition Pipeline

In [3]:
from transformers import pipeline
pipe = pipeline(task="automatic-speech-recognition", model="/content/drive/MyDrive/AFW/AIWispher/large",max_length=5000,device=-1)
pipe("/content/video.wav")

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': " This is when I was 12 years old I went to my grandmother's house with my mom My uncle had a computer at that time which had a lot of games So I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer Then my uncle called me that his car has been punctured There is a tire in the house, we are close Come with the tire When I reached him, I saw that a strange thing was sitting"}

In [ ]:
result = pipe("/content/video.wav")['text']

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
result

" This is when I was 12 years old I went to my grandmother's house with my mom My uncle had a computer at that time which had a lot of games So I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer Then my uncle called me that his car has been punctured There is a tire in the house, we are close Come with the tire When I reached him, I saw that a strange thing was sitting"

## Zeroshot classification using transformers

https://www.section.io/engineering-education/how-to-implement-zero-shot-classification-using-python/#:~:text=Zero-shot%20classification%20is%20a%20technique%20that%20allows%20us,topic%2C%20emotion%2C%20or%20event%20described%20by%20the%20label.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from transformers import pipeline
classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli",device=-1)

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
processor = AutoProcessor.from_pretrained("/content/drive/MyDrive/AFW/AIWispher/large")
model = AutoModelForSpeechSeq2Seq.from_pretrained("/content/drive/MyDrive/AFW/AIWispher/large",return_dict=False)

In [ ]:

nlp= pipeline("automatic-speech-recognition",
                model=model,
                tokenizer=processor,
                framework="pt", return_all_scores=False)

encoded_input = tokenizer(article, truncation=True, max_length=512)
decoded_input = tokenizer.decode(encoded_input["input_ids"], skip_special_tokens = True)
output = nlp(decoded_input)

## Save the zeroshot pipeline

In [ ]:
classifier_pipeline.save_pretrained("/content/drive/MyDrive/AFW/Zeroshot")

In [4]:
result = """ This is when I was 12 years old I went to my grandmother's house with my mom My uncle had a computer at that time which had a lot of games So I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer Then my uncle called me that his car has been punctured There is a tire in the house, we are close Come with the tire When I reached him, I saw that a strange thing was sitting"""

In [5]:
new_pipeline = pipeline ("zero-shot-classification", model = "/content/drive/MyDrive/AFW/Zeroshot")
label_candidate = ['Poistive','Negative','Neutral ']
new_pipeline(result, label_candidate)

{'sequence': " This is when I was 12 years old I went to my grandmother's house with my mom My uncle had a computer at that time which had a lot of games So I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer Then my uncle called me that his car has been punctured There is a tire in the house, we are close Come with the tire When I reached him, I saw that a strange thing was sitting",
 'labels': ['Poistive', 'Negative', 'Neutral '],
 'scores': [0.6192179322242737, 0.3039100170135498, 0.07687200605869293]}

In [ ]:
text = """ This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was."""

In [ ]:
new_pipeline = pipeline ("zero-shot-classification", model = "/content/zeroshot")
label_candidate = ['horror', 'comedy', 'sentimental','romantic','seroius','tragedy']
new_pipeline(text, label_candidate)

{'sequence': " This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was.",
 'labels': ['horror', 'seroius', 'sentimental', 'comedy', 'romantic'],
 'scores': [0.6341521739959717,
  0.18177475035190582,
  0.12192074954509735,
  0.037401482462882996,
  0.02475087158381939]}

In [ ]:
label_candidate = ['Positive', 'Negative', 'Neutral']
classifier_pipeline (translation['text'], label_candidate)

{'sequence': " This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was.",
 'labels': ['Negative', 'Positive', 'Neutral'],
 'scores': [0.520041286945343, 0.24548503756523132, 0.23447363078594208]}

## Huggingface whisper model

In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration,WhisperFeatureExtractor

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
feature = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

In [ ]:
newm = whisper.load_audio('/content/video.wav')

In [ ]:
newm.shape

(731440,)

In [ ]:
translation = newm.transcribe("/content/Audio1.mp3", **translate_options)


In [ ]:
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration,WhisperFeatureExtractor


processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")
processor.save_pretrained("/content/drive/MyDrive/AFW/AIWispher/")
model.save_pretrained("/content/drive/MyDrive/AFW/AIWispher/")

In [ ]:
feat = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")

In [ ]:
feat.save_pretrained("/content/drive/MyDrive/AFW/AIWispher/")

['/content/drive/MyDrive/AFW/AIWispher/preprocessor_config.json']

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-large")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large")

In [ ]:
processor.save_pretrained("/content/whispmodel/")

In [ ]:
model.save_pretrained("whispmodel")

In [ ]:
import whisper
newmodel = whisper.load_model("/content/drive/MyDrive/AFW/AIWispher/large/")

RuntimeError: ignored

In [ ]:
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large']

In [10]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Librosa can handle videos spanning upto 30 seconds

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

speech, _ = librosa.load("/content/video.wav")

processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "en", task = "transcribe")
input_features = processor(speech, return_tensors="pt").input_features 
predicted_ids = model.generate(input_features,max_length=1000)
transcription = processor.batch_decode(predicted_ids)

print(processor.batch_decode(predicted_ids, skip_special_tokens = True))

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[" This is when I was 12 years old, I went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I went there and started playing games. It was night and everyone went to attend the wedding in a nearby village except me, because I wanted to play games on the computer. Then my uncle called me and said that his car's"]


In [ ]:
predicted_ids = model.generate(input_features,max_new_tokens=3000)
transcription = processor.batch_decode(predicted_ids)

print(processor.batch_decode(predicted_ids, skip_special_tokens = True))

[" This is when I was 12 years old, I went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I went there and started playing games. It was night and everyone went to attend the wedding in a nearby village except me, because I wanted to play games on the computer. Then my uncle called me and said that his car's"]


## Inference script

In [ ]:
with open('/content/video.wav', mode='rb') as f:
    response = f.read()

In [ ]:
response

In [6]:
import torch
import whisper
from transformers import pipeline
import torch
import os
import json
import io
import wave

import numpy as np
import scipy.io.wavfile
import soundfile as sf
from scipy.io.wavfile import write

# 
def convert_bytearray_to_wav_ndarray(input_bytearray: bytes, sampling_rate=16000):
    bytes_wav = bytes()
    byte_io = io.BytesIO(bytes_wav)
    write(byte_io, sampling_rate, np.frombuffer(input_bytearray, dtype=np.int16))
    output_wav = byte_io.read()
    output, samplerate = sf.read(io.BytesIO(output_wav))
    return output
   
#

def preprocess_function(data_path,content_type = None ):
    """
    wf = wave.open(data_path, 'rb')
    chunk = 1024
    audio_input = b''
    d = wf.readframes(chunk)
    while len(d) > 0:
        d = wf.readframes(chunk)
        audio_input = audio_input + d
    output = convert_bytearray_to_wav_ndarray(input_bytearray=audio_input)
    scipy.io.wavfile.write("output.wav", 16000, output)
    file_path = "output.wav"
    """
    #file_path = data_path
    return data_path

# 
def predict_function(file_path,model): 
    print("predict function\n")
    asr,nlp = model
    translation = asr(file_path)
    text = translation['text']
    label_candidate = ['positive','negative','neutral']
    result = nlp(text, label_candidate)
    print(result)
    return result
#
def model_load_function(model_file_path):
    pipe = pipeline(task="automatic-speech-recognition", model=model_file_path[0],device=-1)
    nlp_pipeline = pipeline ("zero-shot-classification", model = model_file_path[1],device=-1)
    return (pipe,nlp_pipeline)
#
def postprocess_function(predictions,content_type = None ):
    output = {}
    for k,v in zip(predictions['labels'],predictions['scores']):
        output[k] = round(v * 100.0,2)
        return json.dumps({"response": "Audio Sentiment detected {}".format(output)})

# test the script

if __name__ == "__main__":
    file_name = "video.wav"
    model_file_path = ["/content/drive/MyDrive/AFW/AIWispher/large","/content/drive/MyDrive/AFW/Zeroshot"]
    model = model_load_function(model_file_path)
    input_data = preprocess_function(file_name)
    summary = predict_function(input_data, model)
    output = postprocess_function(summary)
    print(output)
    

 

predict function



/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'sequence': " This is when I was 12 years old I went to my grandmother's house with my mom My uncle had a computer at that time which had a lot of games So I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer Then my uncle called me that his car has been punctured There is a tire in the house, we are close Come with the tire When I reached him, I saw that a strange thing was sitting", 'labels': ['negative', 'neutral', 'positive'], 'scores': [0.8818128705024719, 0.08331611007452011, 0.03487100079655647]}
{"response": "Audio Sentiment detected {'negative': 88.18}"}
